In [2]:
import sys
from pathlib import Path

# Add ../backend to Python path so "import app...." works from notebooks/
sys.path.insert(0, str(Path("..", "backend").resolve()))


In [3]:
import cv2
from pathlib import Path

from app.models.yolo_detector import YOLODetector
from app.tracking.tracker import MultiObjectTracker


In [4]:
video_path = Path("../data/matches/match1.mp4").resolve()
cap = cv2.VideoCapture(str(video_path))

detector = YOLODetector(model_path="yolov8n.pt", conf_th=0.35)
tracker = MultiObjectTracker(iou_threshold=0.3, max_age=15, min_hits=2)

print("Video:", video_path)
print("Opened:", cap.isOpened())



Video: /Users/kevinmathew/Desktop/FotStrike AR/fotstrike-ar/data/matches/match1.mp4
Opened: True


In [5]:
start = 0
end = 200

cap.set(cv2.CAP_PROP_POS_FRAMES, start)

for fidx in range(start, end):
    ok, frame = cap.read()
    if not ok:
        print("Stopped early at frame", fidx)
        break

    detections = detector.detect(frame)
    tracked = tracker.update(detections)

    # Print a small summary every 20 frames
    if fidx % 20 == 0:
        players = [t for t in tracked if t["label"] == "player"]
        balls = [t for t in tracked if t["label"] == "ball"]
        print(f"Frame {fidx}: tracks={len(tracked)} players={len(players)} ball={len(balls)}")

        # Show a few track IDs
        print("  sample IDs:", [t["track_id"] for t in players[:5]], "ball:", [t["track_id"] for t in balls])

cap.release()
print("Done")


Frame 0: tracks=4 players=3 ball=1
  sample IDs: [1, 2, 3] ball: [4]
Frame 20: tracks=7 players=6 ball=1
  sample IDs: [1, 2, 3, 5, 7] ball: [4]
Frame 40: tracks=7 players=6 ball=1
  sample IDs: [1, 2, 3, 7, 9] ball: [4]
Frame 60: tracks=5 players=4 ball=1
  sample IDs: [1, 3, 7, 10] ball: [4]
Frame 80: tracks=9 players=8 ball=1
  sample IDs: [1, 3, 7, 10, 11] ball: [4]
Frame 100: tracks=6 players=5 ball=1
  sample IDs: [1, 3, 10, 16, 17] ball: [19]
Frame 120: tracks=8 players=7 ball=1
  sample IDs: [1, 10, 16, 20, 21] ball: [19]
Frame 140: tracks=15 players=14 ball=1
  sample IDs: [1, 10, 20, 21, 22] ball: [19]
Frame 160: tracks=13 players=12 ball=1
  sample IDs: [1, 20, 21, 25, 29] ball: [19]
Frame 180: tracks=13 players=13 ball=0
  sample IDs: [1, 20, 21, 29, 30] ball: []
Done
